<a href="https://colab.research.google.com/github/Alekhya-pvsns/assignments/blob/master/Lab_5_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

setting up spark environment

Project 1: Machine Learning Project without
Mllib Pipline



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
path_to_spark_archive = '/content/drive/MyDrive/spark-3.0.1-bin-hadoop2.7.tgz'

In [ ]:
!tar xf "{path_to_spark_archive}"

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
!pip install -q findspark

In [ ]:
import findspark

path_to_spark = '/content/spark-3.0.1-bin-hadoop2.7'

findspark.init(path_to_spark)

findspark.find()

'/content/spark-3.0.1-bin-hadoop2.7'

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("Titanic Data") \
        .getOrCreate()

spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName ('Titanic Data') \
        .getOrCreate()

spark

In [ ]:
df = (spark.read
           .format("csv")
           .option("header","true")
           .load("/content/drive/MyDrive/train.csv"))

df.show(5)

In [ ]:
from pyspark.sql.functions import col

dataset = df.select(col('Survived').cast('float'),
                    col('Pclass').cast('float'),
                    col('Sex'),
                    col('Age').cast('float'),
                    col('Fare').cast('float'),
                    col('Embarked')
                    )

dataset.show(4)

In [ ]:
from pyspark.sql.functions import isnull, when, count, col

dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

In [ ]:
dataset = dataset.replace('?', None)\
          .dropna(how='any')

dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

Converting categorical values into numerical values

In [ ]:
dataset.show(3)

In [ ]:
from pyspark.ml.feature import StringIndexer

dataset = StringIndexer(
    inputCol='Sex',
    outputCol='Gender',
    handleInvalid='keep').fit(dataset).transform(dataset)

dataset = StringIndexer(
    inputCol='Embarked',
    outputCol='Boarded',
    handleInvalid='keep').fit(dataset).transform(dataset)

dataset.show(2)

In [ ]:
dataset = dataset.drop('Sex')
dataset = dataset.drop('Embarked')

dataset.show(2)

Feature Engineering

In [ ]:
from pyspark.ml.feature import VectorAssembler

require_featured = ['Pclass', 'Age', 'Fare', 'Gender', 'Boarded']
assembler = VectorAssembler(inputCols=require_featured, outputCol='features')
transformed_data = assembler.transform(dataset)

transformed_data.show(5)

Modeling

In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])
print("Number of train samples: " + str(training_data.count()))
print("Number of test samples:" + str(test_data.count()))

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='Survived',
                            featuresCol='features',
                            maxDepth=5)

model = rf.fit(training_data)
predictions = model.transform(test_data)

Evaluation

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol='Survived',
    predictionCol='prediction',
    metricName = 'accuracy')

accuracy = evaluator.evaluate(predictions)
print('Training Accuracy = ', accuracy)

Training Accuracy =  0.8333333333333334
